In [1]:
import numpy as np
import pandas as pd
import sklearn.preprocessing as skp, StandardScaler
import sklearn.model_selection as skm



ModuleNotFoundError: No module named 'StandardScaler'

In [19]:
df = pd.read_parquet('../data/btcusd_15min_preprcd.parquet')

In [20]:
df.head()
df.columns

Index(['Timestamp', 'open', 'close', 'high', 'low', 'Volume', 'MACD_Histogram',
       'SMA', 'BB_up', 'BB_down', 'RSI', 'OBV', 'CCI', 'pmo', 'R1', 'R2', 'R3',
       'S1', 'S2', 'S3', 'future_price'],
      dtype='object')

In [21]:
df['predict_trend'] = df['future_price'] - df['close']


In [22]:
df["BB_up_diff"] = df["BB_up"] - df["close"]
df["BB_down_diff"] = df["BB_down"] - df["close"]


In [23]:
df_2 = df[[ 'close', 'BB_up_diff', 'BB_down_diff', 'OBV', 'MACD_Histogram', 'RSI', 'CCI', 'predict_trend']]

In [24]:
df_2.to_parquet('../data/btcusd_preprcd_2.parquet')

In [25]:
df['OBV'].describe()



count    442668.000000
mean      57879.677139
std       52510.422053
min      -32962.431276
25%       19153.230006
50%       50940.712963
75%       78210.844338
max      282715.083613
Name: OBV, dtype: float64

In [26]:
def create_obv_zscore(df, window=20):
    df['OBV_mean'] = df['OBV'].rolling(window=window).mean()
    df['OBV_std'] = df['OBV'].rolling(window=20).std()
    df['OBV_Z'] = (df['OBV'] - df['OBV_mean']) / df['OBV_std']
    return df.drop(['OBV_mean', 'OBV_std'], axis=1)
create_obv_zscore(df)['OBV_Z']
df['OBV_Z'].describe()

count    442545.000000
mean          0.011604
std           1.351612
min          -4.248529
25%          -0.985242
50%           0.015530
75%           1.017447
max           4.248529
Name: OBV_Z, dtype: float64

In [27]:
pd.set_option('display.max_columns', None)

df.describe()

,Timestamp,open,close,high,low,Volume,MACD_Histogram,SMA,BB_up,BB_down,RSI,OBV,CCI,pmo,R1,R2,R3,S1,S2,S3,future_price,predict_trend,BB_up_diff,BB_down_diff,OBV_mean,OBV_std,OBV_Z
count,4.426680e+05,442668.000000,442668.000000,442668.000000,442668.000000,442668.000000,442668.000000,442668.000000,442668.000000,442668.000000,442668.000000,442668.000000,442668.000000,442668.000000,442668.000000,442668.000000,442668.000000,442668.000000,442668.000000,442668.000000,442663.000000,442663.000000,442668.000000,442668.000000,442649.000000,442649.000000,442545.000000
mean,1.531118e+09,14968.084098,14968.370617,14999.284478,14934.568755,77.339659,-0.002129,14966.136277,15119.022548,14813.250006,50.920535,57879.677139,3.105265,0.034263,15000.247145,15032.123673,15064.962868,14935.531422,14902.692227,14870.815699,14967.919466,0.547507,150.651931,-155.120611,57881.771703,193.540361,0.011604
std,1.153432e+08,20110.487003,20111.058182,20149.932015,20068.541887,173.367649,30.062748,20106.272088,20308.206918,19906.763409,16.470681,52510.422053,81.175543,1.513084,20151.178194,20191.426863,20233.063772,20069.773746,20028.633693,19988.856306,20109.839043,117.052436,368.542020,374.670867,52509.373905,302.791446,1.351612
min,1.325449e+09,4.140000,4.140000,4.140000,3.800000,0.000000,-826.356587,4.189000,4.250789,3.906975,0.000000,-32962.431276,-231.626413,-63.179916,4.140000,4.140000,4.140000,4.003333,3.056667,-0.320000,4.140000,-6669.066665,-3056.531879,-11190.985868,-31917.968587,0.000000,-4.248529
25%,1.431633e+09,429.000000,429.007500,429.897500,427.987500,8.280887,-1.782181,429.089625,432.562308,425.042127,40.652258,19153.230006,-58.197667,-0.395819,429.950000,430.805000,431.742500,427.992500,426.745833,425.676667,429.067000,-6.028667,1.599002,-142.130708,19155.776955,40.332722,-0.985242
50%,1.531243e+09,6404.830000,6405.000000,6411.655000,6397.715000,28.003769,0.000070,6404.434500,6444.003450,6371.323148,50.623441,50940.712963,6.663014,0.016735,6411.580000,6420.278333,6429.470000,6398.338333,6390.523333,6382.360000,6404.779333,0.000000,20.812183,-22.232146,50936.446518,96.614697,0.015530
75%,1.630860e+09,23807.235000,23809.000000,23852.027500,23767.442500,77.905661,1.771209,23808.325000,24037.636347,23607.585562,61.089692,78210.844338,64.876126,0.462197,23853.068333,23900.302500,23950.920000,23766.020000,23722.247500,23675.083333,23809.463000,6.976333,135.245734,-1.791707,78224.823324,221.899291,1.017447
max,1.737935e+09,108532.000000,108557.000000,109030.000000,108339.000000,10825.114756,777.075797,107930.400000,110440.241570,106699.539214,100.000000,282715.083613,231.626412,96.363636,108928.333333,109496.666667,110782.000000,108237.333333,107942.666667,107885.000000,108244.200000,6293.066669,11230.751230,3321.780073,282353.587632,6735.429761,4.248529


In [ ]:
new_df = df[[ 'Timestamp', 'open', 'high', 'low' ,'close', 'BB_up_diff', 'BB_down_diff', 'OBV_Z', 'MACD_Histogram', 'RSI', 'CCI', 'predict_trend']]

In [29]:
new_df.describe()

,open,high,low,close,BB_up_diff,BB_down_diff,OBV_Z,MACD_Histogram,RSI,CCI,predict_trend
count,442668.000000,442668.000000,442668.000000,442668.000000,442668.000000,442668.000000,442545.000000,442668.000000,442668.000000,442668.000000,442663.000000
mean,14968.084098,14999.284478,14934.568755,14968.370617,150.651931,-155.120611,0.011604,-0.002129,50.920535,3.105265,0.547507
std,20110.487003,20149.932015,20068.541887,20111.058182,368.542020,374.670867,1.351612,30.062748,16.470681,81.175543,117.052436
min,4.140000,4.140000,3.800000,4.140000,-3056.531879,-11190.985868,-4.248529,-826.356587,0.000000,-231.626413,-6669.066665
25%,429.000000,429.897500,427.987500,429.007500,1.599002,-142.130708,-0.985242,-1.782181,40.652258,-58.197667,-6.028667
50%,6404.830000,6411.655000,6397.715000,6405.000000,20.812183,-22.232146,0.015530,0.000070,50.623441,6.663014,0.000000
75%,23807.235000,23852.027500,23767.442500,23809.000000,135.245734,-1.791707,1.017447,1.771209,61.089692,64.876126,6.976333
max,108532.000000,109030.000000,108339.000000,108557.000000,11230.751230,3321.780073,4.248529,777.075797,100.000000,231.626412,6293.066669


In [30]:
new_df.to_parquet('../data/btcusd_precrd_regression.parquet')

In [31]:
df.loc[:, 'predict_trend_classy'] = pd.qcut(
    df['predict_trend'], 
    q=7, 
    labels=[1, 2, 3, 4, 5, 6, 7]
)

In [32]:
new_df.to_parquet('../data/btcusd_ready_classy.parquet')